Datasets:  
[Intel Image Classification | Kaggle](https://www.kaggle.com/puneet6060/intel-image-classification/version/2)  
[Scene Classification | Kaggle](https://www.kaggle.com/nitishabharathi/scene-classification)  

In [1]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth # Connect Drive to Colab
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# Import libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [4]:
# Load and unzip training file
download = drive.CreateFile({'id': '1ZkUvSdKULfGC7U2As_-xaSII6-Y9rhI6'})
download.GetContentFile('train.zip')
!unzip train.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/train/8043.jpg    
  inflating: train/train/12409.jpg   
  inflating: train/train/8145.jpg    
  inflating: train/train/24166.jpg   
  inflating: train/train/12327.jpg   
  inflating: train/train/14516.jpg   
  inflating: train/train/1715.jpg    
  inflating: train/train/18265.jpg   
  inflating: train/train/13421.jpg   
  inflating: train/train/3786.jpg    
  inflating: train/train/12129.jpg   
  inflating: train/train/10873.jpg   
  inflating: train/train/14428.jpg   
  inflating: train/train/23722.jpg   
  inflating: train/train/10028.jpg   
  inflating: train/train/15685.jpg   
  inflating: train/train/17937.jpg   
  inflating: train/train/219.jpg     
  inflating: train/train/22286.jpg   
  inflating: train/train/2230.jpg    
  inflating: train/train/17515.jpg   
  inflating: train/train/19750.jpg   
  inflating: train/train/11237.jpg   
  inflating: train/train/18595.jpg   
  inflating: train/train/19239.jpg   

In [5]:
# Load training data
train = pd.read_csv('train/train.csv')

In [6]:
# Convert list of training images to array
train_image = []
for i in tqdm(range(train.shape[0])): #train.shape[0] => num rows
    # loads image into PIL format at specified target size
    # set grayscale = TRUE if using grayscale images
    img = image.load_img('train/train/'+train['image_name'][i], target_size=(28,28,3))

    # converts PIL image instance to numpy array
    img = image.img_to_array(img)

    # reduce the range of values from 0-255 to 0-1
    img = img/255

    # store image in train_image list
    train_image.append(img)

# converts list to numpy array
X = np.array(train_image)

100%|██████████| 17034/17034 [00:14<00:00, 1153.40it/s]


In [7]:
# one-hot encode target variable

y = train['label'].values # label column
# converts a class vector (integers) to binary class matrix.
y = to_categorical(y)

In [8]:
# Create validation set from training data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

# parameters: 
# test_size = 0.2 of the dataset to include in the test split (default is 0.25)
# random_state ("set seed") controls the shuffling applied to the data before applying the split.

In [9]:
# building a linear stack of layers with the sequential model
model = Sequential()

# convolutional layer
model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(28, 28, 3)))

# convolutional layer
model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(6, activation='softmax'))

In [10]:
# Compile the model created
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [11]:
# Train the model w/ 10 epochs
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 13627 samples, validate on 3407 samples
Epoch 1/10
13627/13627 [==============================] - 13s 964us/step - loss: 1.2108 - accuracy: 0.5207 - val_loss: 0.9180 - val_accuracy: 0.6404
Epoch 2/10
13627/13627 [==============================] - 7s 487us/step - loss: 0.9261 - accuracy: 0.6497 - val_loss: 0.8148 - val_accuracy: 0.7053
Epoch 3/10
13627/13627 [==============================] - 7s 487us/step - loss: 0.7992 - accuracy: 0.7159 - val_loss: 0.6613 - val_accuracy: 0.7623
Epoch 4/10
13627/13627 [==============================] - 7s 486us/step - loss: 0.7196 - accuracy: 0.7495 - val_loss: 0.6255 - val_accuracy: 0.7816
Epoch 5/10
13627/13627 [==============================] - 7s 484us/step - loss: 0.6593 - accuracy: 0.7708 - val_loss: 0.6149 - val_accuracy: 0.7822
Epoch 6/10
13627/13627 [==============================] - 6s 477us/step - loss: 0.5975 - accuracy: 0.7940 - val_loss: 0.6240 - val_accuracy: 0.7678
Epoch 7/10
13627/13627 [==============================] - 7s 4

In [12]:
# Load and unzip test image file
download = drive.CreateFile({'id': '1eiIjK6QszVTeYcjB6E47HXvCQDtP5XYT'})
download.GetContentFile('test.zip')
!unzip test.zip

Archive:  test.zip
   creating: test/
  inflating: test/test.csv           
   creating: test/test/
  inflating: test/test/24076.jpg     
  inflating: test/test/20173.jpg     
  inflating: test/test/21285.jpg     
  inflating: test/test/21435.jpg     
  inflating: test/test/21475.jpg     
  inflating: test/test/21036.jpg     
  inflating: test/test/22570.jpg     
  inflating: test/test/23619.jpg     
  inflating: test/test/22434.jpg     
  inflating: test/test/20090.jpg     
  inflating: test/test/22086.jpg     
  inflating: test/test/23458.jpg     
  inflating: test/test/21934.jpg     
  inflating: test/test/23344.jpg     
  inflating: test/test/22805.jpg     
  inflating: test/test/23136.jpg     
  inflating: test/test/20930.jpg     
  inflating: test/test/23642.jpg     
  inflating: test/test/21485.jpg     
  inflating: test/test/21152.jpg     
  inflating: test/test/23939.jpg     
  inflating: test/test/22457.jpg     
  inflating: test/test/24095.jpg     
  inflating: test/test/204

In [34]:
# Import test file
test = pd.read_csv('test/test.csv')

In [35]:
# Read and store all test images
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('test/test/'+test['file_name'][i], target_size=(28,28,3))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 3000/3000 [00:02<00:00, 1179.82it/s]


In [36]:
test

array([[[[0.3882353 , 0.52156866, 0.67058825],
         [0.38039216, 0.5137255 , 0.65882355],
         [0.3764706 , 0.50980395, 0.654902  ],
         ...,
         [0.09803922, 0.05490196, 0.07843138],
         [0.40784314, 0.35686275, 0.33333334],
         [0.08235294, 0.07843138, 0.09803922]],

        [[0.39215687, 0.5254902 , 0.67058825],
         [0.39607844, 0.5294118 , 0.6745098 ],
         [0.39215687, 0.5254902 , 0.67058825],
         ...,
         [0.21960784, 0.19607843, 0.07058824],
         [0.2784314 , 0.25490198, 0.20784314],
         [0.19607843, 0.19607843, 0.15686275]],

        [[0.40784314, 0.5372549 , 0.6745098 ],
         [0.4       , 0.5254902 , 0.6745098 ],
         [0.38431373, 0.50980395, 0.65882355],
         ...,
         [0.06666667, 0.08235294, 0.11764706],
         [0.1764706 , 0.16862746, 0.17254902],
         [0.14901961, 0.12941177, 0.11764706]],

        ...,

        [[0.33333334, 0.43137255, 0.5176471 ],
         [0.32156864, 0.42745098, 0.5019608 ]

In [41]:
# making predictions of classes
prediction = model.predict_classes(test)

In [16]:
model.save('model.h5')

In [22]:
saved_model = keras.models.load_model('model.h5')

In [47]:
a = []
b = image.load_img('test/test/22513.jpg', target_size=(28,28,3))
b = image.img_to_array(img)
b = b/255
a.append(b)
a = np.array(a)
a

array([[[[2.5528644e-03, 2.6912726e-03, 2.8296809e-03],
         [2.5528644e-03, 2.6912726e-03, 2.8296809e-03],
         [2.5682431e-03, 2.7066513e-03, 2.8450596e-03],
         ...,
         [2.8911957e-03, 2.9988466e-03, 3.1218762e-03],
         [2.9065744e-03, 3.0142253e-03, 3.1372549e-03],
         [2.9219531e-03, 2.9988466e-03, 3.0911188e-03]],

        [[2.6451366e-03, 2.7527874e-03, 2.8758170e-03],
         [2.5990005e-03, 2.7066513e-03, 2.8296809e-03],
         [2.6297579e-03, 2.7374087e-03, 2.8604383e-03],
         ...,
         [3.0142253e-03, 3.1218762e-03, 3.2756634e-03],
         [2.8450596e-03, 2.9527105e-03, 3.0757401e-03],
         [7.6893506e-05, 1.3840832e-04, 1.8454441e-04]],

        [[2.7066513e-03, 2.8143022e-03, 2.9373318e-03],
         [2.6912726e-03, 2.7989235e-03, 2.9219531e-03],
         [2.7220300e-03, 2.8296809e-03, 2.9527105e-03],
         ...,
         [2.9680892e-03, 3.0757401e-03, 3.1987696e-03],
         [3.4909651e-03, 3.5371012e-03, 3.6139947e-03],
  

In [48]:
saved_model.predict_classes(a)

array([5])